In [8]:
%cd embedding-level/
%load_ext autoreload
%autoreload 2

[Errno 2] No such file or directory: 'embedding-level/'
/home/rhuangbi/Desktop/dev/AI-Safety_SCAV/embedding-level
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from instructions import *

dataset_name = 'Demo'
model_nickname = 'llama2-7b'
classifier_type = 'safety'

insts = load_instructions_by_size(
    dataset_name=dataset_name,
    label_list=["Malicious", "Safe"],
    train_size=0.5,
)

In [10]:
from model_extraction import ModelExtraction

llm = ModelExtraction(model_nickname)

pos_train_embds = llm.extract_embds(insts['train'][0])
neg_train_embds = llm.extract_embds(insts['train'][1])
pos_test_embds = llm.extract_embds(insts['test'][0])
neg_test_embds = llm.extract_embds(insts['test'][1])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting embeddings: 50it [00:02, 18.80it/s]
Extracting embeddings: 50it [00:01, 27.59it/s]
Extracting embeddings: 50it [00:01, 35.32it/s]
Extracting embeddings: 50it [00:01, 37.24it/s]


In [13]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch
from reduction import Reduction

# 行和列数
c, r = 4, 8

# 创建子图
fig = make_subplots(
    rows=c, cols=r,
    subplot_titles=[f'Layer {i * r + j}' for i in range(c) for j in range(r)]
)

for i in range(c):
    for j in range(r):
        layer = i * r + j
        pca = Reduction(2)

        # 准备数据
        train_data = torch.vstack([pos_train_embds.layers[layer], neg_train_embds.layers[layer]])
        pca.fit(train_data)

        pos_train_pca = pca.transform(pos_train_embds.layers[layer])
        neg_train_pca = pca.transform(neg_train_embds.layers[layer])
        pos_test_pca = pca.transform(pos_test_embds.layers[layer])
        neg_test_pca = pca.transform(neg_test_embds.layers[layer])

        # 当前子图行列索引
        row, col = i + 1, j + 1

        # 添加到对应子图
        fig.add_trace(
            go.Scatter(
                x=pos_train_pca[:, 0], y=pos_train_pca[:, 1],
                mode='markers',
                marker=dict(color='red', opacity=0.3),
                name='malicious_train',
                text=insts['train'][0],  # 文字数组
                hoverinfo='text',  # 显示文字
                showlegend=(i == 0 and j == 0)  # 只在第一个子图显示图例
            ),
            row=row, col=col
        )
        fig.add_trace(
            go.Scatter(
                x=neg_train_pca[:, 0], y=neg_train_pca[:, 1],
                mode='markers',
                marker=dict(color='blue', opacity=0.3),
                name='safe_train',
                text=insts['train'][1],
                hoverinfo='text',
                showlegend=(i == 0 and j == 0)
            ),
            row=row, col=col
        )
        fig.add_trace(
            go.Scatter(
                x=pos_test_pca[:, 0], y=pos_test_pca[:, 1],
                mode='markers',
                marker=dict(color='orange', opacity=0.3),
                name='malicious_test',
                text=insts['test'][0],
                hoverinfo='text',
                showlegend=(i == 0 and j == 0)
            ),
            row=row, col=col
        )
        fig.add_trace(
            go.Scatter(
                x=neg_test_pca[:, 0], y=neg_test_pca[:, 1],
                mode='markers',
                marker=dict(color='green', opacity=0.3),
                name='safe_test',
                text=insts['test'][1],
                hoverinfo='text',
                showlegend=(i == 0 and j == 0)
            ),
            row=row, col=col
        )

# 设置布局
fig.update_layout(
    title="PCA Projection of Embeddings Across Layers",
    height=c * 300,  # 根据行数动态调整高度
    width=r * 300,   # 根据列数动态调整宽度
    showlegend=True
)

# 显示图表
fig.show()

/cm/shared/apps/Anaconda3/2023.09-0/lib/python3.11/site-packages/sklearn/decomposition/_pca.py:640: RuntimeWarning:

invalid value encountered in divide

